In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

url_path = r"https://www.google.com/maps"
driver.get(url_path)

cities = ["Mississauga", "Oakville", "Milton", "Burlington", "Hamilton", "Grimsby", "St-Catharines", "Niagara Falls"]
for city in cities:
    search_input = f"Property Managers in {city}"
    xpath_searchbox = r'//*[@id="searchboxinput"]'
    enter = driver.find_element(By.XPATH, xpath_searchbox)
    enter.send_keys(search_input)
    enter.send_keys(Keys.ENTER)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, "pane")))

    # Function to scroll down a specified area
    def scroll_down(driver, element_xpath, pixels=500):
        scroll_element = driver.find_element(By.XPATH, element_xpath)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_element)
        time.sleep(2)

    result_xpath_base = '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{}]/div/a'
    names = []
    urls = []
    latitudes = []
    longitudes = []
    addresses = []
    contacts = []

    # Main loop to extract data
    odd = 3
    count = 0
    while True:
        result_xpath = result_xpath_base.format(odd)
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, result_xpath)))
            driver.find_element(By.XPATH, result_xpath).click()

            try:
                name = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1').text
            except:
                name = "N/A"
            names.append(name)
            
            url = driver.current_url
            urls.append(url)
            
            try:
                coordinates = url.split("@")[1].split(",")
                latitude = float(coordinates[0])
                longitude = float(coordinates[1].split("z")[0])
            except:
                latitude = longitude = "N/A"
            latitudes.append(latitude)
            longitudes.append(longitude)

            # Scroll into view the address and contact
            scroll_down(driver, '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]', 1000)

            try:
                address = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[7]/div[3]/button/div/div[2]/div[1]').text
            except:
                address = "N/A"
            addresses.append(address)

            try:
                contact = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[7]/div[5]/button/div/div[2]/div[1]').text
            except:
                contact = "N/A"
            contacts.append(contact)

            # Go back to the search results
            driver.back()
            wait.until(EC.presence_of_element_located((By.XPATH, result_xpath_base.format(odd))))

            count += 1
            odd += 2
        except Exception as e:
            print(f"Exception: {e}")
            break

        # Scroll down to ensure more results are loaded
        scroll_down(driver, '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]', 2500)
        time.sleep(1)

    # Create DataFrame
    df = pd.DataFrame({
        "Name": names,
        "URL": urls,
        "Latitude": latitudes,
        "Longitude": longitudes,
        "Address": addresses,
        "Contact": contacts
    })

    # Save DataFrame to CSV
    path = f'Product_managers_in_{city}.csv'
    df.to_csv(path, index=False)

# Close the driver
driver.quit()

Exception: Message: 
Stacktrace:
	GetHandleVerifier [0x00D4C113+48259]
	(No symbol) [0x00CDCA41]
	(No symbol) [0x00BD0A17]
	(No symbol) [0x00C10BED]
	(No symbol) [0x00C10C9B]
	(No symbol) [0x00C4BC12]
	(No symbol) [0x00C30DE4]
	(No symbol) [0x00C49B9C]
	(No symbol) [0x00C30B36]
	(No symbol) [0x00C0570D]
	(No symbol) [0x00C062CD]
	GetHandleVerifier [0x010065A3+2908435]
	GetHandleVerifier [0x01043BBB+3159851]
	GetHandleVerifier [0x00DE50CB+674875]
	GetHandleVerifier [0x00DEB28C+699900]
	(No symbol) [0x00CE6244]
	(No symbol) [0x00CE2298]
	(No symbol) [0x00CE242C]
	(No symbol) [0x00CD4BB0]
	BaseThreadInitThunk [0x75F17BA9+25]
	RtlInitializeExceptionChain [0x76EBC10B+107]
	RtlClearBits [0x76EBC08F+191]

Exception: Message: 
Stacktrace:
	GetHandleVerifier [0x00D4C113+48259]
	(No symbol) [0x00CDCA41]
	(No symbol) [0x00BD0A17]
	(No symbol) [0x00C10BED]
	(No symbol) [0x00C10C9B]
	(No symbol) [0x00C4BC12]
	(No symbol) [0x00C30DE4]
	(No symbol) [0x00C49B9C]
	(No symbol) [0x00C30B36]
	(No symbol) 

In [28]:
df

,Name,URL,Latitude,Longitude,Address,Contact
0,N/A,https://www.google.com/maps/search/flower+shop...,30.189317,71.447803,N/A,N/A
1,N/A,https://www.google.com/maps/search/flower+shop...,30.189562,71.447802,N/A,N/A
2,The Florist & Decorators,https://www.google.com/maps/place/Fresh+Blosso...,30.213120,71.327929,N/A,N/A
3,Fresh Blossoms,https://www.google.com/maps/place/Bismillah+Ki...,30.213120,71.327929,N/A,N/A
4,Bismillah Kit Flower Shop,https://www.google.com/maps/search/flower+shop...,30.189317,71.447803,N/A,N/A
5,Al-Madina Cute Flowers Shop & Drink Corner,https://www.google.com/maps/place/Al-Madina+Cu...,30.213120,71.327929,N/A,N/A
6,The Florist and Decorator,https://www.google.com/maps/place/Hassan+Flowe...,30.213120,71.327929,N/A,N/A
7,Hassan Flower shop and Decoration center.,https://www.google.com/maps/search/flower+shop...,30.189317,71.447803,N/A,N/A
8,خالد فلاور شاپ,https://www.google.com/maps/search/flower+shop...,30.189317,71.447803,N/A,N/A
9,Fresh bloom's,https://www.google.com/maps/search/flower+shop...,30.189317,71.447803,N/A,N/A
